In [12]:
#implement our proposed method --- multitasks framework
import numpy as np
import tensorflow as tf
import deepchem as dc
import pandas as pd

In [13]:
#load train dataset and test dataset
train_dataset_file = "train_dataset.csv"
test_dataset_file = "test_dataset.csv"

In [14]:
#get the list of all GPCR names in input file
df = pd.read_csv('Input_Pos10Neg10.csv')
df1 = df.drop(['ChEMBL_ID', 'Smiles'], axis = 1) 
GPCR_tasks=list(df1.columns.values)

In [15]:
loader = dc.data.CSVLoader(tasks=GPCR_tasks, smiles_field="Smiles", featurizer=dc.feat.CircularFingerprint(size=1024))

In [16]:
#feature enginnering, convert each ligand (in train set)into 1024 dimensional vector using ECFP4 method
train_dataset = loader.featurize(train_dataset_file)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from train_dataset.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
TIMING: featurizing shard 0 took 15.640 s
TIMING: dataset construction took 15.943 s
Loading dataset from disk.


In [17]:
#feature enginnering, convert each ligand (in test set)into 1024 dimensional vector using ECFP4 method
test_dataset = loader.featurize(test_dataset_file)

Loading raw samples now.
shard_size: 8192
About to start loading CSV from test_dataset.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 3.820 s
TIMING: dataset construction took 3.916 s
Loading dataset from disk.


In [18]:
n_tasks = len(GPCR_tasks) ##25 tasks (GPCRS)
n_features = train_dataset.get_data_shape()[0] ##1024 features

In [19]:
model = dc.models.MultitaskClassifier(n_tasks, n_features)

In [20]:
model.fit(train_dataset)

0.0001489030562100067

In [21]:
#get the true value in test set
y_true = test_dataset.y
#get the prediction value from the multitask framework
y_pred = model.predict(test_dataset)
#using ROC score to evaluate the prediction
metric = dc.metrics.roc_auc_score

In [22]:
for i in range(n_tasks):
    score = metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
    print(GPCR_tasks[i], score)

5ht1d_rat 0.9973208884279665
ackr3_human 0.9943216274326039
ccr1_human 0.9897499184073107
ccr1_mouse 0.9815375302663438
ccr2_human 0.978529403934723
ccr2_mouse 0.9474604622871046
ccr3_human 0.9923931784346409
ccr3_mouse 0.9995166163141995
ccr3_rat 0.9996980676328502
ccr4_human 0.9940026487157847
ccr5_human 0.9765016457572206
ccr5_mouse 0.9790655339805825
ccr6_human 0.9608088478520372
ccr8_human 1.0
ccr9_human 1.0
ccrl2_human 0.9998686974789917
cx3c1_human 0.9997493824066449
cxcr1_human 0.9858170357200582
cxcr2_human 0.9971567506577665
cxcr3_human 0.9964376876876877
cxcr3_mouse 0.9940837378640777
cxcr3_rat 0.9963811821471653
cxcr4_human 0.9986660879323304
cxcr5_human 0.9992920711974109
q9jly8_rat 0.9978394261515858


In [53]:
#ablation study, change the size of hidden layer
def optimize_layers(layer1, layer2):
    score_comb = []
    mod = dc.models.MultitaskClassifier(n_tasks, n_features, layer_sizes = [layer1, layer2])
    mod.fit(train_dataset)
    y_pred_mod = mod.predict(test_dataset)
    for i in range(n_tasks):
        score = metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred_mod[:,i])
        score_comb.append(score)
    return score_comb

In [85]:
score1 = np.round(optimize_layers(1000,50),4)
score2 = np.round(optimize_layers(1000,100),4)
score3 = np.round(optimize_layers(1000,150),4)
score4 = np.round(optimize_layers(2000,50),4)
score5 = np.round(optimize_layers(2000,100),4)
score6 = np.round(optimize_layers(2000,150),4)
score7 = np.round(optimize_layers(3000,50),4)
score8 = np.round(optimize_layers(3000,100),4)
score9 = np.round(optimize_layers(3000,150),4)

In [117]:
##summarize the prediction results using different layer size in multitasks framework
result = pd.DataFrame(np.vstack((score1,score2,score3,score4,score5,score6,score7,score8,score9)))
result = result.T

In [115]:
#result = result.rename(columns = {0:"GPCR", 1:"(1000,50)", 2:"(1000,100)", 3:"(1000,150)", 4:"(2000,50)", 5:"(2000,100)", 6:"(2000,150)", 7:"(3000,50)", 8:"(3000,100)", 9:"(3000,150)"})
result

,0,1,2,3,4,5,6,7,8
0,0.8719,0.9488,0.8976,0.9588,0.9500,0.9652,0.6936,0.9259,0.9812
1,0.9330,0.9843,0.9194,0.9454,0.9788,0.9768,0.8401,0.8915,0.9769
2,0.9819,0.9841,0.9887,0.9673,0.9877,0.9858,0.8392,0.9679,0.9430
3,0.9848,0.9870,0.9896,0.9896,0.9961,0.9782,0.9331,0.9651,0.9941
4,0.9633,0.9392,0.9555,0.9572,0.9500,0.9647,0.9372,0.9729,0.9595
5,0.9367,0.9707,0.9668,0.9569,0.9714,0.9752,0.9050,0.9675,0.9611
6,0.9855,0.9811,0.9893,0.9915,0.9879,0.9909,0.9725,0.9839,0.9903
7,0.9977,0.9994,0.9961,0.9886,0.9954,0.9981,0.9836,0.9917,0.9930
8,0.9995,0.9997,0.9997,1.0000,0.9997,0.9998,0.9819,0.9997,0.9903
9,0.9882,0.9879,0.9878,0.9467,0.9583,0.9895,0.9475,0.9873,0.9804


In [119]:
#calculate the mean AUC value of multitasks frame with diffent layer size
result.mean(axis = 0)

0    0.978240
1    0.983236
2    0.979604
3    0.973652
4    0.980868
5    0.986004
6    0.931016
7    0.977124
8    0.976072
dtype: float64

In [120]:
#calculate the median AUC value of multitasks frame with diffent layer size
result.median(axis = 0)

0    0.9882
1    0.9879
2    0.9896
3    0.9828
4    0.9877
5    0.9909
6    0.9475
7    0.9847
8    0.9812
dtype: float64